In [1]:
import sys
from pathlib import Path
import shutil
import json
import csv

from shapely.geometry import Point
import requests

DIR = Path('..')
sys.path.append(str(DIR))

import wavetrace as wt

TMP = DIR/'tmp'
IN_DIR = TMP/'inputs'
OUT_DIR = TMP/'outputs'

%load_ext autoreload
%autoreload 2



In [16]:
import math

6371*math.pi/180


111.19492664455873

In [2]:
# # Visually check that SRTM tiles cover NZ by 
# # making polygons and viewing them at https://geojson.io.

# tids = wt.SRTM_NZ_TILE_IDS
# collection = {
#     'type': 'FeatureCollection', 
#     'features': list(map(wt.build_feature, tids))
#     }
# json.dumps(collection)

In [3]:
# # Download SRTM data from NASA

# path = Path('/home/araichev/srtmnz/srtm1')
# url = "http://e4ftl01.cr.usgs.gov/SRTM/SRTMGL3.003/2000.02.11/"
# #url = "http://e4ftl01.cr.usgs.gov/SRTM/SRTMGL1.003/2000.02.11/"
# tids = wt.SRTM_NZ_TILE_IDS
# for i, tid in enumerate(sorted(tids)):
#     filename = tid + '.SRTMGL3.hgt.zip'
#     href = url + filename
#     print(i, href)

In [4]:
# # Download elevation test data

# TEST_DATA_DIR = DIR/'tests'/'data'
# ts = wt.read_transmitters(TEST_DATA_DIR/'transmitters.csv')[:2]
# lon_lats = wt.get_lon_lats(ts)
# tiles = wt.get_tile_names(lon_lats)
# print('Tiles to download =', tiles)
# wt.download_srtm(tiles, TEST_DATA_DIR/'srtm3', api_key=wt.get_secret('GITLAB_API_KEY'))

In [5]:
# Start fresh
wt.rm_paths(TMP)


In [12]:
high_definition = False

path = DIR/'tests'/'data'/'transmitters.csv'
transmitters = wt.read_transmitters(path)
print('num transmitters =', len(transmitters))
for t in transmitters:
    print(t)

num transmitters = 20
{'vertical_beamwidth': '', 'power_eirp': 4.0, 'site_name': '4', 'antenna_height': 10.0, 'name': 'GrayWireless_4', 'horizontal_beamwidth': '', 'longitude': 174.336547, 'polarization': 0.0, 'downtilt': '', 'frequency': 5725.0, 'latitude': -35.712247, 'network_name': 'Gray Wireless', 'bearing': '0'}
{'vertical_beamwidth': '30', 'power_eirp': 4.0, 'site_name': '5', 'antenna_height': 20.0, 'name': 'MyNetwork_5', 'horizontal_beamwidth': '90', 'longitude': 174.281534, 'polarization': 0.0, 'downtilt': '1', 'frequency': 5725.0, 'latitude': -35.658841, 'network_name': 'My Network', 'bearing': '0'}
{'vertical_beamwidth': '30', 'power_eirp': 4.0, 'site_name': '6', 'antenna_height': 20.0, 'name': 'MyNetwork_6', 'horizontal_beamwidth': '90', 'longitude': 173.156248, 'polarization': 0.0, 'downtilt': '1', 'frequency': 5725.0, 'latitude': -34.819217, 'network_name': 'My Network', 'bearing': '0'}
{'vertical_beamwidth': '30', 'power_eirp': 4.0, 'site_name': '7', 'antenna_height': 20

In [13]:
# Create SPLAT transmitter files

in_path = DIR/'tests'/'data'/'transmitters.csv'
out_path = OUT_DIR/'splat_files'
if out_path.exists():
    shutil.rmtree(str(out_path))
    
wt.create_splat_transmitter_files(in_path, out_path)

% ls {out_path}

CanUCMe_21.az       MyNetwork_11.az   MyNetwork_9.az   Skynet_14.az
CanUCMe_21.el       MyNetwork_11.el   MyNetwork_9.el   Skynet_14.el
CanUCMe_21.lrp      MyNetwork_11.lrp  MyNetwork_9.lrp  Skynet_14.lrp
CanUCMe_21.qth      MyNetwork_11.qth  MyNetwork_9.qth  Skynet_14.qth
CanUCMe_22.az       MyNetwork_5.az    RuralCom_18.az   Skynet_15.az
CanUCMe_22.el       MyNetwork_5.el    RuralCom_18.el   Skynet_15.el
CanUCMe_22.lrp      MyNetwork_5.lrp   RuralCom_18.lrp  Skynet_15.lrp
CanUCMe_22.qth      MyNetwork_5.qth   RuralCom_18.qth  Skynet_15.qth
CanUCMe_23.az       MyNetwork_6.az    RuralCom_19.az   Skynet_16.az
CanUCMe_23.el       MyNetwork_6.el    RuralCom_19.el   Skynet_16.el
CanUCMe_23.lrp      MyNetwork_6.lrp   RuralCom_19.lrp  Skynet_16.lrp
CanUCMe_23.qth      MyNetwork_6.qth   RuralCom_19.qth  Skynet_16.qth
GrayWireless_4.az   MyNetwork_7.az    Skynet_12.az     Skynet_17.az
GrayWireless_4.el   MyNetwork_7.el    Skynet_12.el     Skynet_17.el
GrayWireless_4.lrp  MyNetwork_7.lrp   Skyn

In [14]:
# Download topography data

buffer = 0.5 # decimal degrees
points = [Point(p).buffer(buffer) for p in wt.get_lonlats(transmitters)]
tids = wt.compute_intersecting_tiles(list(points))
print('Tiles to download =', tids)

if high_definition:
    path = IN_DIR/'srtm1'
else:
    path = IN_DIR/'srtm3'
api_key = wt.get_secret('GITLAB_API_KEY')

wt.download_srtm(tids, path, high_definition=high_definition, api_key=api_key)

% ls {path}

Tiles to download = ['S35E172', 'S35E173', 'S36E173', 'S36E174', 'S37E173', 'S37E174', 'S37E175', 'S38E174', 'S38E175', 'S38E176', 'S38E177', 'S39E174', 'S39E175', 'S39E176', 'S40E175', 'S40E176', 'S40E177', 'S41E174', 'S41E175', 'S41E176', 'S42E174', 'S42E175', 'S42E176', 'S45E168', 'S45E169', 'S46E168', 'S46E169', 'S47E167', 'S47E168', 'S48E167', 'S48E168']


KeyboardInterrupt: 

In [9]:
# Create SPLAT topography files

if high_definition:
    in_path = IN_DIR/'srtm1'
else:
    in_path = IN_DIR/'srtm3'
out_path = OUT_DIR/'splat_files'

wt.create_splat_topography_files(in_path, out_path, high_definition=high_definition)

% ls {out_path}

-35:-34:186:187-hd.sdf  CanUCMe_23.qth      MyNetwork_7.qth  Skynet_13.qth
-36:-35:185:186-hd.sdf  GrayWireless_4.az   MyNetwork_8.az   Skynet_14.az
-36:-35:186:187-hd.sdf  GrayWireless_4.el   MyNetwork_8.el   Skynet_14.el
-37:-36:184:185-hd.sdf  GrayWireless_4.lrp  MyNetwork_8.lrp  Skynet_14.lrp
-37:-36:185:186-hd.sdf  GrayWireless_4.qth  MyNetwork_8.qth  Skynet_14.qth
-38:-37:183:184-hd.sdf  MyNetwork_10.az     MyNetwork_9.az   Skynet_15.az
-38:-37:184:185-hd.sdf  MyNetwork_10.el     MyNetwork_9.el   Skynet_15.el
-38:-37:185:186-hd.sdf  MyNetwork_10.lrp    MyNetwork_9.lrp  Skynet_15.lrp
-40:-39:183:184-hd.sdf  MyNetwork_10.qth    MyNetwork_9.qth  Skynet_15.qth
-41:-40:184:185-hd.sdf  MyNetwork_11.az     RuralCom_18.az   Skynet_16.az
-42:-41:184:185-hd.sdf  MyNetwork_11.el     RuralCom_18.el   Skynet_16.el
-46:-45:191:192-hd.sdf  MyNetwork_11.lrp    RuralCom_18.lrp  Skynet_16.lrp
-47:-46:192:193-hd.sdf  MyNetwork_11.qth    RuralCom_18.qth  Skynet_16.qth
CanUCMe_21.az           MyNetwo

In [10]:
# Create coverage reports

in_path = OUT_DIR/'splat_files'
out_path = OUT_DIR/'coverage_reports'
wt.compute_coverage(in_path, out_path, transmitters=transmitters, 
  high_definition=high_definition)

% ls {out_path}

Timing compute_coverage...
2016-08-18 18:04:39.403941   Began process
2016-08-19 05:32:30.330512   Finished in 687.83 min
CanUCMe_21-ck.ppm               MyNetwork_9-ck.ppm
CanUCMe_21.kml                  MyNetwork_9.kml
CanUCMe_21.ppm                  MyNetwork_9.ppm
CanUCMe_21-site_report.txt      MyNetwork_9-site_report.txt
CanUCMe_22-ck.ppm               RuralCom_18-ck.ppm
CanUCMe_22.kml                  RuralCom_18.kml
CanUCMe_22.ppm                  RuralCom_18.ppm
CanUCMe_22-site_report.txt      RuralCom_18-site_report.txt
CanUCMe_23-ck.ppm               RuralCom_19-ck.ppm
CanUCMe_23.kml                  RuralCom_19.kml
CanUCMe_23.ppm                  RuralCom_19.ppm
CanUCMe_23-site_report.txt      RuralCom_19-site_report.txt
GrayWireless_4-ck.ppm           Skynet_12-ck.ppm
GrayWireless_4.kml              Skynet_12.kml
GrayWireless_4.ppm              Skynet_12.ppm
GrayWireless_4-site_report.txt  Skynet_12-site_report.txt
MyNetwork_10-ck.ppm             Skynet_13-ck.ppm
MyNetwork

In [11]:
# Post-process coverage reports

path = OUT_DIR/'coverage_reports'
wt.postprocess_coverage(path)

% ls {path}

CanUCMe_21-ck.png               MyNetwork_9-ck.png
CanUCMe_21.kml                  MyNetwork_9.kml
CanUCMe_21.png                  MyNetwork_9.png
CanUCMe_21-site_report.txt      MyNetwork_9-site_report.txt
CanUCMe_21.tif                  MyNetwork_9.tif
CanUCMe_22-ck.png               RuralCom_18-ck.png
CanUCMe_22.kml                  RuralCom_18.kml
CanUCMe_22.png                  RuralCom_18.png
CanUCMe_22-site_report.txt      RuralCom_18-site_report.txt
CanUCMe_22.tif                  RuralCom_18.tif
CanUCMe_23-ck.png               RuralCom_19-ck.png
CanUCMe_23.kml                  RuralCom_19.kml
CanUCMe_23.png                  RuralCom_19.png
CanUCMe_23-site_report.txt      RuralCom_19-site_report.txt
CanUCMe_23.tif                  RuralCom_19.tif
GrayWireless_4-ck.png           Skynet_12-ck.png
GrayWireless_4.kml              Skynet_12.kml
GrayWireless_4.png              Skynet_12.png
GrayWireless_4-site_report.txt  Skynet_12-site_report.txt
GrayWireless_4.tif              Skyn